In [35]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler

#import cleaned_data.csv to pandas.

data = pd.read_csv('cleaned_data.csv')


In [36]:
data

,CreditScore,Age,Balance,NumOfProducts,HasCrCard,EstimatedSalary,Geography,Gender
0,581,34.0,148882.54,1,1.0,84560.88,France,Male
1,588,36.0,131778.58,1,1.0,136024.31,Germany,Male
2,593,30.0,144772.69,1,1.0,29792.11,France,Female
3,678,37.0,138476.41,1,1.0,106851.60,Spain,Male
4,583,40.0,81274.33,1,1.0,170843.07,Germany,Male
...,...,...,...,...,...,...,...,...
125519,752,32.0,160729.10,1,1.0,142223.94,Spain,Male
125520,667,43.0,83765.35,1,1.0,118358.54,Germany,Female
125521,570,29.0,116099.82,1,1.0,148087.62,Spain,Male
125522,575,36.0,178032.53,1,1.0,42181.68,France,Female


In [37]:
# Define the features that need encoding and those that don't
categorical_features = ['Geography', 'Gender']
numerical_features = ['CreditScore', 'Age', 'Balance', 'NumOfProducts','HasCrCard', 'EstimatedSalary']


In [38]:
# Create transformers for the pipeline
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Adding RobustScaler for numerical features
numerical_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())
])


In [39]:
# Combine transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])


In [40]:
# Example pipeline: let's say we just want to transform the data
transform_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Apply the pipeline to the data
transformed_data = transform_pipeline.fit_transform(data)


In [41]:
transformed_data

array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.40941262],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.22369007],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -1.08317758],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
         0.        ,  0.37209281],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.93076119],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -1.15184176]])